# Retrieval-Augmented Generation (RAG)

In [1]:
import chromadb
import dotenv
from pathlib import Path
from agents import Agent, Runner, function_tool, trace

dotenv.load_dotenv()

True

Create a static calorie table that we can use as a tool:

In [3]:
# We populated the RAG with the data from the data/calories.csv file in
# the rag_setup.ipynb notebook
chroma_client = chromadb.PersistentClient("../chroma")
nutrition_db = chroma_client.get_collection(name="nutrition_db")

In [4]:
results = nutrition_db.query(query_texts=["banana"], n_results=2)
for i, doc in enumerate(results["documents"][0]):
    print(sorted(results["metadatas"][0][i].items()))
    print(doc)
    print("\n")

/Users/venkir/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:03<00:00, 27.4MiB/s]


[('calories_per_100g', 89.0), ('food_category', 'fruits'), ('food_item', 'banana'), ('keywords', 'banana_fruits'), ('kj_per_100g', 374.0), ('serving_info', '100g')]
Food: Banana
        Category: Fruits
        Nutritional Information:
        - Calories: 89 per 100g
        - Energy: 374 kJ per 100g
        - Serving size reference: 100g

        This is a fruits food item that provides 89 calories per 100 grams.


[('calories_per_100g', 50.0), ('food_category', '(fruit)juices'), ('food_item', 'banana juice'), ('keywords', 'banana_juice_(fruit)juices'), ('kj_per_100g', 210.0), ('serving_info', '100ml')]
Food: Banana Juice
        Category: (Fruit)Juices
        Nutritional Information:
        - Calories: 50 per 100g
        - Energy: 210 kJ per 100g
        - Serving size reference: 100ml

        This is a (fruit)juices food item that provides 50 calories per 100 grams.




In [12]:
# Assignment: By Venki
# We populated the RAG with the data from the data/nutrition_qna file in
# the rag_setup.ipynb notebook
chroma_client = chromadb.PersistentClient("../chroma")
nutrition_qna = chroma_client.get_collection(name="nutrition_qna")

In [14]:
results = nutrition_qna.query(query_texts=["pregnancy"], n_results=2)
for i, doc in enumerate(results["documents"][0]):
    print(sorted(results["metadatas"][0][i].items()))
    print(doc)
    print("\n")

[('is_pregnancy', True)]
Question: What are some possible physical changes that a pregnant woman could experience in the middle months of gestation?
        Answer: During weeks 13-27, you may see an increase in weight and feel more hunger. Backaches might occur frequently, along with leg cramps and heartburn.

        This Q&A pair provides information about nutrition and health topics.


[('is_pregnancy', True)]
Question: What health issues can make a pregnancy more challenging?
        Answer: There are several medical conditions that can potentially increase the risk associated with Pregnancy. These include Anemia during this stage, Hypertensive Disorders related to gestation, Diabetes Mellitus coexisting with Pregnancy, Obesity during pregnancy, as well as Adolescent or Teenage pregnancies.

        This Q&A pair provides information about nutrition and health topics.




In [9]:
@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to look up calorie information for specific food items, but not for meals.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = nutrition_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Nutrition Information:\n" + "\n".join(formatted_results)

In [20]:
@function_tool
def nutrition_qna_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to look up answers for specific topic.

    Args:
        query: The topic item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = nutrition_qna.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition Q&A information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        formatted_results.append(doc)
        
    return "Related answers to your question:\n" + "\n".join(formatted_results)
        


Let's test this out: 

_The following cell only works before you add the `@function_tool` annotation to `calorie_lookup_tool` function_

In [8]:
calorie_lookup_tool('bananas')

'Nutrition Information:\nBanana (Fruits): 89.0 calories per 100g\nBanana Juice ((Fruit)Juices): 50.0 calories per 100g\nBanana Nut Bread (Pastries,Breads&Rolls): 326.0 calories per 100g'

In [19]:
nutrition_qna_tool('pregnancy')

"Related answers to your question:\nQuestion: What are some possible physical changes that a pregnant woman could experience in the middle months of gestation?\n        Answer: During weeks 13-27, you may see an increase in weight and feel more hunger. Backaches might occur frequently, along with leg cramps and heartburn.\n\n        This Q&A pair provides information about nutrition and health topics.\nQuestion: What health issues can make a pregnancy more challenging?\n        Answer: There are several medical conditions that can potentially increase the risk associated with Pregnancy. These include Anemia during this stage, Hypertensive Disorders related to gestation, Diabetes Mellitus coexisting with Pregnancy, Obesity during pregnancy, as well as Adolescent or Teenage pregnancies.\n\n        This Q&A pair provides information about nutrition and health topics.\nQuestion: What are some hormonal changes that take place in a mother's body during pregnancy?\n        Answer: During preg

In [21]:
calorie_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful nutrition assistant giving out calorie information and nutrtion advice    .
    You give concise answers.
    
    If you need to look up calorie information, use the calorie_lookup_tool.
    If are asked a question about nutrition, always use the nutrition_qna_tool first to see if there is an answer in the knowledge base.
    """,
    tools=[calorie_lookup_tool, nutrition_qna_tool],
)

In [22]:
with trace("Nutrition Assistant with Nutrition and Calorie RAG"):
    result = await Runner.run(
        calorie_agent,
        "What are the best meal choices for pregnant women and how many calories do they have?",
    )
    print(result.final_output)

Here’s a concise guide based on the knowledge base:

- Calorie guidance: about 200 extra calories per day during pregnancy, on top of a balanced diet.

- Key foods to include daily:
  - Iron-rich foods: lean meats, fish, eggs, beans, lentils, fortified cereals.
  - Calcium-rich foods: dairy or fortified plant milks, yogurt, cheese; leafy greens.
  - Folate: leafy greens, citrus, beans, fortified grains.
  - Protein: lean meats, fish (low-mercury), eggs, dairy, legumes, nuts.
  - Whole grains and fiber: whole grains, fruits, vegetables.
  - Vitamin C foods to aid iron absorption: citrus, peppers, berries.
  - Hydration and a variety of fruits/vegetables.

- Example meal ideas (~200–400 kcal per item, adjust to your daily plan):
  - Breakfast: yogurt with berries and a sprinkle of granola (~250–350 kcal).
  - Lunch: whole-grain wrap with grilled chicken, spinach, and avocado (~350–500 kcal).
  - Snack: apple with peanut butter (~150–250 kcal) or a small smoothie with yogurt and greens (~